<a href="https://colab.research.google.com/github/m10singh94/CatDogClassification/blob/main/CatDogClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cat and Dog Classification

We will create a simple `NN model` and a `ViT model` to detect the class of image (either a dog or a cat), and try to demonstrate a demo of the model in Gradio in HuggingFace Space.

## Starting with Importing all the files
* torch
* torchvision
* matplotlib
* torchinfo


In [ ]:
# importing torch and torchvision
try:
    import torch
    import torchvision
    assert int(torch.__version__.split(".")[1]) >= 12, "torch version should be 1.12+"
    assert int(torchvision.__version__.split(".")[1]) >= 13, "torchvision version should be 0.13+"
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")
except:
    print(f"[INFO] torch/torchvision versions not as required, installing nightly versions.")
    !pip3 install -U torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
    import torch
    import torchvision
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")

import matplotlib.pyplot as plt
from torchinfo import summary
from torch import nn
from torchvision import datasets, transforms

from pathlib import Path
import os
import zipfile
import requests